## osumapper #4: New Map Reader


Set the input file string to a timed (having the right BPM/offset) .osu file.

It converts the map/music to Python readable format.

In [6]:
from act_newmap_prep import *

# input file here! (don't remove the "r" before string)
file_path = r'..\..\test_data\test.osu'

# Or use auto timing with music file only!!

# from act_timing import *;
# music_path = r"..\..\test_data\audio.mp3"
# file_path = get_timed_osu_file(music_path, game_mode=0);

step4_read_new_map(file_path);

## osumapper #5: Rhythm Predictor

Calculates a map's rhythm based on the music and timing.

Parameters:

"note_density" determines how many notes will be placed on the timeline, ranges from 0 to 1.<br>
"slider_favor" determines how the model favors sliders against circles, ranges from -1 to 1.<br>
"dist_multiplier" determines the distance snap. ranges from 0 to +∞. Of course 0/+∞ are not advisable.<br>
"divisor_favor" determines how the model favors notes to be on X divisors starting from a beat (white, blue, red, blue), ranges from -1 to 1 each.<br>
"slider_max_ticks" determines the max amount of time a slider can slide, ranges from 1 to +∞.

In [7]:
from act_rhythm_calc import *

model = step5_load_model();
npz = step5_load_npz();
params = step5_set_params(dist_multiplier=1, note_density=0.35, slider_favor=0, divisor_favor=[0] * 4, slider_max_ticks=8);

predictions = step5_predict_notes(model, npz, params);
converted = step5_convert_sliders(predictions, params);

Model not trained! Trying default model...
402 notes predicted.


In [8]:
step5_save_predictions(converted);

## osumapper #6: Map flow generator

Generate the final map using a Generative Adversarial Network.

Parameters:

- note_distance_basis: the baseline for distance snap between notes
- max_ticks_for_ds: max number of time ticks (each 1/4) that it uses the distance snap
- next_from_slider_end: use slider end instead of slider head for calculating distance
- box_loss_border, box_loss_value: it's like a barrier on the map edges that bounces off the circles
- divisor, note_group_size: don't change unless you're using a special model built for it
- good_epoch, max_epoch: controls the training time. less time makes it faster but risks less quality
- g_\*, c_\*: hyperparameters used by GAN. No one knows how they work but they mysterically affect the result

In [9]:
from act_gan import *;

gan_params = {
    "divisor" : 4,
    "good_epoch" : 12,
    "max_epoch" : 30,
    "note_group_size" : 10,
    "g_epochs" : 1,
    "c_epochs" : 1,
    "g_batch" : 50,
    "g_input_size" : 50,
    "c_true_batch" : 140,
    "c_false_batch" : 5,
    "c_randfalse_batch" : 5,
    "note_distance_basis" : 200,
    "next_from_slider_end" : False,
    "max_ticks_for_ds" : 1,
    "box_loss_border" : 0.1,
    "box_loss_value" : 0.4,
    "box_loss_weight" : 1
};

step6_set_gan_params(gan_params);
osu_a, data = step6_run_all();

Flow dataset not found! Trying default model...
# of groups: 30
Group 0, Epoch 1: G loss: 0.4887359142303467 vs. C loss: 0.17373043298721313
Group 0, Epoch 2: G loss: 0.155124694108963 vs. C loss: 0.08615109324455261
Group 0, Epoch 3: G loss: 0.042548246681690216 vs. C loss: 0.07827290147542953
Group 0, Epoch 4: G loss: 0.029634976759552956 vs. C loss: 0.08010450750589371
Group 0, Epoch 5: G loss: 0.026263469830155373 vs. C loss: 0.08030789345502853
Group 0, Epoch 6: G loss: 0.027061041444540024 vs. C loss: 0.07927156239748001
Group 0, Epoch 7: G loss: 0.042396098375320435 vs. C loss: 0.07785363495349884
Group 0, Epoch 8: G loss: 0.08727593719959259 vs. C loss: 0.07211453467607498
Group 0, Epoch 9: G loss: 0.17904192209243774 vs. C loss: 0.07341364771127701
Group 0, Epoch 10: G loss: 0.16221490502357483 vs. C loss: 0.07056011259555817
Group 0, Epoch 11: G loss: 0.12331415712833405 vs. C loss: 0.07142826914787292
Group 0, Epoch 12: G loss: 0.14549346268177032 vs. C loss: 0.0635052099823

Group 8, Epoch 9: G loss: 0.005939131136983633 vs. C loss: 0.08279314637184143
Group 8, Epoch 10: G loss: 0.004309249110519886 vs. C loss: 0.08275222033262253
Group 8, Epoch 11: G loss: 0.005535080097615719 vs. C loss: 0.08263321965932846
Group 8, Epoch 12: G loss: 0.0064623975194990635 vs. C loss: 0.08246049284934998
Group 9, Epoch 1: G loss: 0.5228552222251892 vs. C loss: 0.20669786632061005
Group 9, Epoch 2: G loss: 0.21791572868824005 vs. C loss: 0.09122256934642792
Group 9, Epoch 3: G loss: 0.04944026097655296 vs. C loss: 0.07746871560811996
Group 9, Epoch 4: G loss: 0.014171699993312359 vs. C loss: 0.08180190622806549
Group 9, Epoch 5: G loss: 0.00859748013317585 vs. C loss: 0.08241061121225357
Group 9, Epoch 6: G loss: 0.007788853254169226 vs. C loss: 0.08262456953525543
Group 9, Epoch 7: G loss: 0.008763390593230724 vs. C loss: 0.08269762992858887
Group 9, Epoch 8: G loss: 0.00819123350083828 vs. C loss: 0.08266081660985947
Group 9, Epoch 9: G loss: 0.007041799835860729 vs. C l

Group 17, Epoch 5: G loss: 0.015766052529215813 vs. C loss: 0.07796741276979446
Group 17, Epoch 6: G loss: 0.014767855405807495 vs. C loss: 0.06886859983205795
Group 17, Epoch 7: G loss: 0.020272929221391678 vs. C loss: 0.060358721762895584
Group 17, Epoch 8: G loss: 0.025711432099342346 vs. C loss: 0.06256294995546341
Group 17, Epoch 9: G loss: 0.01592596061527729 vs. C loss: 0.06208685413002968
Group 17, Epoch 10: G loss: 0.020427372306585312 vs. C loss: 0.05326857417821884
Group 17, Epoch 11: G loss: 0.018700435757637024 vs. C loss: 0.04984404891729355
Group 17, Epoch 12: G loss: 0.01550397276878357 vs. C loss: 0.06019081547856331
Group 18, Epoch 1: G loss: 0.5207778811454773 vs. C loss: 0.20130300521850586
Group 18, Epoch 2: G loss: 0.2681790292263031 vs. C loss: 0.09715154767036438
Group 18, Epoch 3: G loss: 0.049507997930049896 vs. C loss: 0.07925970107316971
Group 18, Epoch 4: G loss: 0.011822800152003765 vs. C loss: 0.08057930320501328
Group 18, Epoch 5: G loss: 0.0077455267310

Group 25, Epoch 12: G loss: 0.13572315871715546 vs. C loss: 0.07789414376020432
Group 26, Epoch 1: G loss: 0.5204063653945923 vs. C loss: 0.20222517848014832
Group 26, Epoch 2: G loss: 0.21703919768333435 vs. C loss: 0.09360025823116302
Group 26, Epoch 3: G loss: 0.03362772613763809 vs. C loss: 0.07948652654886246
Group 26, Epoch 4: G loss: 0.00962607841938734 vs. C loss: 0.08202505856752396
Group 26, Epoch 5: G loss: 0.007594266440719366 vs. C loss: 0.08263426274061203
Group 26, Epoch 6: G loss: 0.00875058863312006 vs. C loss: 0.08279234915971756
Group 26, Epoch 7: G loss: 0.00977818388491869 vs. C loss: 0.08292774856090546
Group 26, Epoch 8: G loss: 0.0060012368485331535 vs. C loss: 0.08291272819042206
Group 26, Epoch 9: G loss: 0.0048163775354623795 vs. C loss: 0.08292245119810104
Group 26, Epoch 10: G loss: 0.0025778147391974926 vs. C loss: 0.08282873779535294
Group 26, Epoch 11: G loss: 0.0028496752493083477 vs. C loss: 0.08273813128471375
Group 26, Epoch 12: G loss: 0.00237081758

### Since the generation will take a while...

we can appreciate a nice picture of Cute Sophie!!

<img src="https://i.imgur.com/Ko2wogO.jpg" />

Do a little modding to the map.

Parameters:

- stream_regularizer: fix bad streams. integer for modes (0,1,2,3,4) 0=inactive
- slider_mirror: mirror slider ends if they go outside map area. (0,1) 0=inactive 1=active

In [ ]:
from act_modding import *

modding_params = {
    "stream_regularizer" : 1,
    "slider_mirror" : 1
}

osu_a, data = step7_modding(osu_a, data, modding_params);

Finally, save the data into an .osu file!

In [ ]:
from act_final import *

saved_osu_name = step8_save_osu_file(osu_a, data);

# for taiko mode only (comment out the above line and use below)
# from act_taiko_hitsounds import *
# taiko_hitsounds_params = step8_taiko_hitsounds_set_params(divisor=4, metronome_count=4)
# hitsounds = step8_apply_taiko_hitsounds(osu_a, data, params=taiko_hitsounds_params)
# saved_osu_name = step8_save_osu_file(osu_a, data, hitsounds=hitsounds);

# clean up the folder
step8_clean_up();

If it works alright, you should have a nice .osu file under the folder of these notebooks now!

If it does not work, please tell me the problem so probably I could fix it!

For bug reports and feedbacks either report it on github or use discord: <br>
[https://discord.com/invite/npmSy7K](https://discord.com/invite/npmSy7K)